# Figure 3



In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib
import matplotlib as mpl


mpl.rcParams.update({
    'font.family':          'sans-serif',
    'font.sans-serif':      ['Arial'],
    'mathtext.fontset':     'custom',
    'mathtext.rm':          'Arial',
    'mathtext.it':          'Arial:italic',
    'mathtext.bf':          'Arial:bold',
    'mathtext.cal':         'Arial',        # for calligraphic if you like
})

In [2]:
pop_df=pd.read_csv(r'C:\Users\Yasaman\Downloads\World_bank_population.csv',skiprows=3)
pop_df=pop_df[['Country Code','2019']].dropna()
pop_df['2019']=pop_df['2019'].astype(int)
possible_countries=pop_df.query(" `2019` >=1000000")['Country Code'].values
possible_countries=[x.lower() for x in possible_countries]


excluded_iso3_codes = [
    "IRL",  # Ireland
    "SSD",  # South Sudan
    "SDN",  # Sudan
    "COG",  # Republic of the Congo
    "COD",  # Democratic Republic of the Congo
    "GIN",  # Guinea
    "GNB",  # Guinea-Bissau
    "GNQ",  # Equatorial Guinea
    "PNG",  # Papua New Guinea
    "XKX",  # Kosovo (unofficial)
    "MNE",  # Montenegro
    "SRB",  # Serbia
    "TLS",   # Timor-Leste
    'GEO'

]
excluded_iso3_codes=[c.lower() for c in excluded_iso3_codes]
possible_countries=list(set(possible_countries)-set(excluded_iso3_codes))

In [3]:
matplotlib.rc('font', family='Arial') 

In [4]:
df=pd.read_csv(r"C:\Users\Yasaman\Downloads\Attention-fractional counting.csv")
df=df[df['country'].isin(possible_countries)]

df=df.rename(columns={'aggregated_value':'count', 'country':'Mention_country', 'affiliation_country':'Aff_country'})

df=df[df['year'].isin(np.arange(2002, 2020, 1))]
Country_list={'Egypt':'EGY', 'Tunisia':'TUN','Libya':'LBY','Syria':'SYR','Yemen':'YEM','Bahrain':'BHR','Jordan':'JOR','Kuwait':'KWT','Morocco':'MAR','Oman':'OMN'}
rev_Country_list={Country_list[key]: key for key in Country_list}
abbr=[country.lower() for country in Country_list.values()]
physical_sciences=[ 'MATH', 'ENGI', 'PHYS', 'COMP','MULT']
df=df[~df['subjarea'].isin(physical_sciences)]

In [5]:
def find_cat(subject):  
    social_sciences=['PSYC', 'SOCI', 'DECI', 'ARTS', 'ECON', 'BUSI']
    health_sciences=['MEDI', 'NURS', 'DENT', 'HEAL', 'VETE']
    physical_sciences=['CHEM', 'EART', 'ENER', 'CENG',  'ENVI','MATE']
    life_sciences=['PHAR', 'NEUR', 'AGRI', 'IMMU', 'BIOC']
    if subject in social_sciences:
        return 'Social sciences'
    elif  subject in health_sciences:
        return 'Health sciences'
    elif subject in physical_sciences:
        return 'Physical sciences'
    elif subject in life_sciences:
        return 'Life sciences'
    else:
        return subject

In [6]:
df['cat']=df['subjarea'].apply(find_cat)
df['year_arabspring']=df['year'].apply(lambda x: 'before' if x>=2002 and x<=2010 else 'after' )
df['is_internal_attention']=df['Mention_country']==df['Aff_country']

result_df=df.groupby(['cat','Mention_country', 'year_arabspring', 'is_internal_attention'])['count'].sum().reset_index()

total_df=result_df.groupby(['Mention_country','year_arabspring','is_internal_attention'])['count'].sum().reset_index().rename(columns={'count':'total'})


result_df=result_df.merge(total_df, on=['Mention_country','year_arabspring','is_internal_attention'])
result_df['Share']=result_df['count']/result_df['total']

In [12]:
# Add brackets and messages
def add_bracket(axis, x_start, x_end, y, text, ratio=0.01):
    """Add a bracket with text between two x positions."""
    axis.plot([x_start, x_start, x_end, x_end], [y, y + 1*ratio, y + 1*ratio, y], color="black", lw=1.5)
    axis.text((x_start + x_end) / 2, y +1.5*ratio, text, ha='center', va='bottom', fontsize=10)

Country_list = {'Egypt':'EGY', 'Tunisia':'TUN', 'Libya':'LBY', 'Syria':'SYR', 'Yemen':'YEM',
                'Bahrain':'BHR', 'Jordan':'JOR', 'Kuwait':'KWT', 'Morocco':'MAR', 'Oman':'OMN'}
rev_Country_list = {Country_list[key].lower(): key for key in Country_list}
country_names = list(Country_list.keys())
custom_colors = [
    "#E69F00",  # Orange
    "#56B4E9",  # Sky Blue
    "#009E73",  # Bluish Green
    "#6929C4",  # Purple 70
    "#CC79A7",  # Reddish Purple

    "#0072B2",  # Blue
    "#D55E00",  # Vermillion
    # Three additional color‑blind‑friendly hues from Paul Tol’s “Muted” scheme:
    "#999933",  # Teal
    "#8A3800",  # Magenta 50
    "#882255",  # Wine (added)
]

fig, ax = plt.subplots(nrows=2, ncols=4, figsize=(11, 6), dpi=1500)
subplot_labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
k=0
for j, is_internal in enumerate([True, False]):
    for i, category in enumerate(['Social sciences', 'Health sciences', 'Life sciences', 'Physical sciences']):
        subset = result_df.query(f'cat == "{category}" and Mention_country in @abbr and is_internal_attention == {is_internal}')
        subset['Mention_country'] = subset['Mention_country'].map(rev_Country_list)
        subset['sorted_order'] = subset['Mention_country'].apply(lambda x: country_names.index(x))
        subset = subset.sort_values('sorted_order').drop(columns=['sorted_order'])

        # Reset the index if needed
        subset.reset_index(drop=True, inplace=True)
        x_labels = subset['Mention_country'].unique()
        x =  np.array([0,1,3,4,5,7, 8, 9, 10, 11] ) # X positions for the bars
        
        width = 0.4  # Width of each bar
        before_data = subset[subset['year_arabspring'] == 'before']['Share']
        after_data = subset[subset['year_arabspring'] == 'after']['Share']
        
        # Bars for 'before'
        ax[j][i].bar(
            x - width / 2, 
            before_data, 
            width=width, 
            color=custom_colors, 
            label='Before'
        )
        
        # Bars for 'after' with dashed patterns
        ax[j][i].bar(
            x + width / 2, 
            after_data, 
            width=width, 
            color='none', 
            edgecolor=custom_colors, 
            linewidth=1.5, 
            hatch='//', 
            label='After'
        )
        medians=result_df.query(f'cat == "{category}" and is_internal_attention=={is_internal}').groupby('year_arabspring')['Share'].median().to_dict()

        ax[j][i].axhline(medians['before'], color='k')
        ax[j][i].axhline(medians['after'], color='k', linestyle='--')
        
        # Set x-axis labels and title
        ax[j][i].set_xticks(x)
        ax[j][i].set_xticklabels(x_labels, rotation=90)
        ax[j][i].set_title(category)
        if j==0:
            ax[j][i].set_ylim(0, 0.85)
            ax[j][0].set_ylabel('Domestic attention relative share')
            add_bracket(ax[j][i], -0.5, 2, 0.75, "GO")
            add_bracket(ax[j][i], 2.5, 6,  0.75, "CW")
            add_bracket(ax[j][i], 7, 11.5,  0.75, "GC")

        else:
            ax[j][i].set_ylim(0, 0.75)
            ax[j][0].set_ylabel('Foriegn attention relative share')
            add_bracket(ax[j][i], -0.5, 2, 0.675, "GO")
            add_bracket(ax[j][i], 2.5, 6,  0.675, "CW")
            add_bracket(ax[j][i], 7, 11.5,  0.675, "GC")

        if i !=0:
            ax[j][i].set_yticklabels([])
            ax[j][i].text(-0.05, 1.05, subplot_labels[k].lower(), transform=ax[j][i].transAxes, fontsize=15, fontweight='bold', va='center', ha='center')
        else:
            ax[j][i].text(-0.05, 1.05, subplot_labels[k].lower(), transform=ax[j][i].transAxes, fontsize=15, fontweight='bold', va='center', ha='center')
        
        # Add subplot label (A, B, C, D) to the top-left corner
        
        
        k+=1

# Create custom legend handles for the boxes
solid_box = mpatches.Patch(facecolor='k', edgecolor='k', label='Before Arabspring')  # Solid box
dashed_box = mpatches.Patch(facecolor='white', edgecolor='k', hatch='////', label='After Arabspring')  # Dashed pattern inside
    # Create custom legend handles with grey color
before_handle = mlines.Line2D([], [], color='k', label='World\'s median before')
after_handle = mlines.Line2D([], [], color='k', label='World\'s median after', linestyle='--')




        # Add the legend with custom handles
lgd=fig.legend(handles=[solid_box, dashed_box, before_handle,after_handle], loc='lower center', frameon=False, ncol=4,bbox_to_anchor=(0.5, -0.05))

plt.tight_layout()
fig.savefig('Figure4.pdf', bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()


C:\Users\Yasaman\AppData\Local\Temp\ipykernel_31136\3763356857.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['Mention_country'] = subset['Mention_country'].map(rev_Country_list)
C:\Users\Yasaman\AppData\Local\Temp\ipykernel_31136\3763356857.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['sorted_order'] = subset['Mention_country'].apply(lambda x: country_names.index(x))
C:\Users\Yasaman\AppData\Local\Temp\ipykernel_31136\3763356857.py:32: SettingWithCopyWarning: 
A value is trying 

In [11]:
result_df[(result_df['Mention_country']=='syr')&(result_df['cat']=='Social sciences')]

,cat,Mention_country,year_arabspring,is_internal_attention,count,total,Share
2256,Social sciences,syr,after,False,1188.324163,1843.927824,0.644453
2257,Social sciences,syr,after,True,38.388948,501.811549,0.076501
2258,Social sciences,syr,before,False,199.955525,489.944571,0.408119
2259,Social sciences,syr,before,True,14.561884,259.765367,0.056058


In [10]:
result_df[(result_df['Mention_country']=='lby')&(result_df['cat']=='Social sciences')]

,cat,Mention_country,year_arabspring,is_internal_attention,count,total,Share
2055,Social sciences,lby,after,False,541.052724,1096.141306,0.493598
2056,Social sciences,lby,after,True,40.578690,258.648211,0.156888
2057,Social sciences,lby,before,False,62.583918,315.303485,0.198488
2058,Social sciences,lby,before,True,7.052778,166.069597,0.042469
